In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import *
from keras.backend import tf as ktf
from keras.callbacks import EarlyStopping
from tqdm import tqdm

from keras.datasets import mnist
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
import csv
%matplotlib inline

In [ ]:
data = pd.read_csv('train.csv')
train, dev = train_test_split(data, test_size=0.25,random_state=0, stratify=data['label'])

Y_train = train.iloc[:, 0].as_matrix()
Y_train = np.eye(10, dtype='float32')[Y_train] 
#it's imp to specify dtype, as default is float64, but placeholder expects float32.
X_train = train.iloc[:,1:].as_matrix().astype('float32')
Y_dev = dev.iloc[:, 0].as_matrix()
Y_dev = np.eye(10, dtype='float32')[Y_dev]
X_dev = dev.iloc[:,1:].as_matrix().astype('float32')
X_test = pd.read_csv('test.csv').as_matrix().astype('float32')

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_dev = X_dev.reshape(X_dev.shape[0], 28, 28, 1)



print(train.shape, dev.shape)
print(X_train.shape, Y_train.shape)
print(X_dev.shape, Y_dev.shape)
print(X_test.shape)

In [ ]:
# X_test = np.multiply(X_test, 1.0/255.0)
# X_train = np.multiply(X_train, 1.0/255.0)
# X_dev = np.multiply(X_dev, 1.0/255.0)

In [ ]:
batch_size = 64
input_shape = (28, 28, 1)

In [ ]:
gen = ImageDataGenerator(
        rotation_range=8,
        width_shift_range=0.08,
        shear_range=0.3,
        height_shift_range=0.08,
        zoom_range=0.08)

train_gen = gen.flow(X_train, Y_train, batch_size=batch_size)
dev_gen = gen.flow(X_dev, Y_dev, batch_size=batch_size)

In [ ]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [ ]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1,)
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(' accuracy')


    plt.subplot(1, 2, 2)
    
    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.legend(['train', 'val'], loc='upper left')
    plt.title('loss')
    plt.show()

In [ ]:
def getModel():
    model = Sequential()
    model.add(Lambda(norm_input, input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
model = getModel()

In [ ]:
history1 = model.fit_generator(train_gen, steps_per_epoch=train_gen.n, epochs=1, validation_data=dev_gen, validation_steps=dev_gen.n)

In [ ]:
model.optimizer.lr=0.1

In [ ]:
history2 = model.fit_generator(train_gen, steps_per_epoch=train_gen.n, epochs=4, validation_data=dev_gen, validation_steps=dev_gen.n)

In [ ]:
plot_training(history2)

In [ ]:
model.optimizer.lr=0.01

In [ ]:
history3 = model.fit_generator(train_gen, steps_per_epoch=train_gen.n, epochs=12, validation_data=dev_gen, validation_steps=dev_gen.n)

In [ ]:
plot_trainingtraining_training(history3)

In [ ]:
model.optimizer.lr=0.001

In [ ]:
history4 = model.fit_generator(train_gen, steps_per_epoch=train_gen.n, epochs=1, validation_data=dev_gen, validation_steps=dev_gen.n)

In [ ]:
model.save_weights('models/bn+do+da.h5')

In [ ]:
Y_predicted = model.predict(X_test)

In [ ]:
pred = pd.read_csv('sample_submission.csv')
pred['Label'] = Y_predicted
pred.to_csv('submission-cnn-keras-v2.csv', index=False)

## Training on whole training dataset for final submission

In [ ]:
Y_data = data.iloc[:, 0].as_matrix()
Y_data = np.eye(10, dtype='float32')[Y_data] 
X_data = data.iloc[:,1:].as_matrix().astype('float32')
X_data = X_data.reshape(X_data.shape[0], 28, 28, 1)
print(X_data.shape, Y_data.shape)

In [ ]:
data_gen = gen.flow(X_data, Y_data, batch_size=32) #once I used batch_size=8, ended up with poor acc of .97

In [ ]:
model = getModel()

In [ ]:
whistory1 = model.fit_generator(data_gen, steps_per_epoch=data_gen.n, epochs=4)

In [ ]:
model.optimizer.lr=0.1

In [ ]:
history5 = model.fit_generator(data_gen, steps_per_epoch=train_gen.n, epochs=4)

In [ ]:
model.optimizer.lr=0.01

In [ ]:
history6 = model.fit_generator(data_gen, steps_per_epoch=train_gen.n, epochs=12)

In [ ]:
model.optimizer.lr=0.001

In [ ]:
history7 = model.fit_generator(data_gen, steps_per_epoch=train_gen.n, epochs=1)

In [ ]:
Y_predicted = model.predict(X_test, verbose=1)

In [ ]:
pred = pd.read_csv('sample_submission.csv')
pred['Label'] = np.argmax(Y_predicted, axis=1)
pred.to_csv('submission-cnn-keras-v4.csv', index=False)

In [ ]:
model.save_weights('models/kv1-total.h5')